In [10]:
import numpy as np
import os
import librosa
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import StandardScaler



#in conda environment:
#*. Open appropriate anaconda prompt(according to environment) and try following commands inorder,

#First, conda install -c numba numba

#Then, conda install -c conda-forge librosa

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping

AttributeError: module 'keras.backend' has no attribute 'eager'

In [25]:
filelist_train = os.listdir('Train\Sound') 
filelist_test = os.listdir('DataSet\Test\LibriSpeech/test-other') 

train_df = pd.DataFrame(filelist_train)
test_df = pd.DataFrame(filelist_train)
val_df = pd.DataFrame(filelist_train)



In [26]:
print(test_df)

                        0
0     116-288045-0000.wav
1     116-288045-0001.wav
2     116-288045-0002.wav
3     116-288045-0003.wav
4     1255-74899-0000.wav
..                    ...
127   8254-84205-0003.wav
128  8288-274150-0000.wav
129  8288-274150-0001.wav
130  8288-274150-0002.wav
131  8288-274150-0003.wav

[132 rows x 1 columns]


In [27]:
# Renaming the column name to file
train_df = train_df.rename(columns={0:'file'})
test_df = test_df.rename(columns={0:'file'})
val_df  = val_df.rename(columns={0:'file'})

# Code in case we have to d0rop the '.DS_Store' and reset the index
train_df[train_df['file']=='.DS_Store']
train_df.drop(16, inplace=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)

test_df[test_df['file']=='.DS_Store']
test_df.drop(16, inplace=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

val_df[val_df['file']=='.DS_Store']
val_df.drop(16, inplace=True)
val_df = val_df.sample(frac=1).reset_index(drop=True)

In [28]:
# We create an empty list where we will append all the speakers ids for each row of our dataframe by slicing the file name since we know the id is the first number before the hash
speaker = []
for i in range(0, len(train_df)):
    speaker.append(train_df['file'][i].split('-')[0])
# We now assign the speaker to a new column 
train_df['speaker'] = speaker

speaker = []
for i in range(0, len(test_df)):
    speaker.append(test_df['file'][i].split('-')[0])
# We now assign the speaker to a new column 
test_df['speaker'] = speaker

speaker = []
for i in range(0, len(val_df)):
    speaker.append(val_df['file'][i].split('-')[0])
# We now assign the speaker to a new column 
val_df['speaker'] = speaker

In [29]:
print(train_df.head())
print(test_df.head())
print(val_df.head())



                  file speaker
0  6599-38590-0003.wav    6599
1  3915-57461-0001.wav    3915
2  4831-18525-0002.wav    4831
3  1255-74899-0001.wav    1255
4  4323-13259-0001.wav    4323
                   file speaker
0  1686-142278-0003.wav    1686
1   6599-38590-0003.wav    6599
2   700-122866-0003.wav     700
3   4323-13259-0002.wav    4323
4   5543-27761-0003.wav    5543
                   file speaker
0   8254-84205-0001.wav    8254
1  3663-172005-0002.wav    3663
2  1651-136854-0003.wav    1651
3  8173-294714-0000.wav    8173
4   4572-64670-0003.wav    4572


In [57]:
def extract_features(files):
    # Sets the name to be the path to where the file is in my computer
    file_name = os.path.join(os.path.abspath('Train\Sound')+'/'+str(files.file))
    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))
        # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return (mfccs, chroma, mel, contrast, tonnetz)

In [60]:
train_features = train_df.apply(extract_features, axis=1)
print(train_features)
print(train_features.shape)
print(len(train_features))

features_train = []
for i in range(0, len(train_features)):
    features_train.append(np.concatenate((
        train_features[i][0],
        train_features[i][1], 
        train_features[i][2], 
        train_features[i][3],
        train_features[i][4]), axis=0))

X_train = np.array(features_train)

file       6599-38590-0003.wav
speaker                   6599
Name: 0, dtype: object
e:\skola\2021-2022\LP3\D7041E\D7041E\project\Train\Sound/6599-38590-0003.wav


C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/678629343.py:16: FutureWarning: Pass y=[-1.0050000e-03 -1.1443967e-03 -9.1314997e-04 ...  2.8472254e-04
 -1.6279630e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


file       3915-57461-0001.wav
speaker                   3915
Name: 1, dtype: object
e:\skola\2021-2022\LP3\D7041E\D7041E\project\Train\Sound/3915-57461-0001.wav


C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/678629343.py:16: FutureWarning: Pass y=[-0.00135608 -0.00509374 -0.00791924 ...  0.00140722  0.00144452
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


file       4831-18525-0002.wav
speaker                   4831
Name: 2, dtype: object
e:\skola\2021-2022\LP3\D7041E\D7041E\project\Train\Sound/4831-18525-0002.wav


...

file       7601-101619-0002.wav
speaker                    7601
Name: 129, dtype: object
e:\skola\2021-2022\LP3\D7041E\D7041E\project\Train\Sound/7601-101619-0002.wav


C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/678629343.py:16: FutureWarning: Pass y=[ 2.5980102e-04 -1.8611642e-04 -1.8095872e-03 ... -4.0874638e-05
 -5.3156527e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


file       3660-6517-0002.wav
speaker                  3660
Name: 130, dtype: object
e:\skola\2021-2022\LP3\D7041E\D7041E\project\Train\Sound/3660-6517-0002.wav


C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/678629343.py:16: FutureWarning: Pass y=[2.2389844e-05 3.5969628e-05 3.9027469e-05 ... 1.2863637e-03 1.9227307e-03
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


0      ([-388.6012, 139.31332, -74.48864, 61.75871, 1...
1      ([-329.02042, 127.05577, -18.469053, 41.294617...
2      ([-334.54926, 106.47782, -26.942184, 51.45521,...
3      ([-335.9633, 68.584, -21.299364, 55.912136, -3...
4      ([-396.2699, 101.6964, -20.676378, 19.699017, ...
                             ...                        
126    ([-333.76083, 104.48733, -37.793816, 34.950558...
127    ([-304.52036, 111.68174, -31.81777, 32.650272,...
128    ([-348.44696, 104.391655, -14.983462, 52.5148,...
129    ([-373.1289, 110.39732, -16.45454, 51.548775, ...
130    ([-348.76874, 132.81535, -30.076817, 60.098576...
Length: 131, dtype: object
(131,)
131


In [66]:
print(train_features[0])
print('-------------------------------------')
print(len(X_train[0]))

(array([-3.8860120e+02,  1.3931332e+02, -7.4488640e+01,  6.1758709e+01,
        1.9636963e+01, -9.3895893e+00,  2.2494841e+01, -2.2333488e+01,
       -1.2205078e+01,  7.7036762e+00,  3.4415815e+00,  1.0439122e+00,
       -4.6982021e+00,  4.8353972e+00,  4.1715817e+00,  1.4601759e+00,
        2.4572465e+00, -2.6237324e-01,  1.7318016e+00, -2.7708554e+00,
       -1.5413499e+00,  1.5889698e+00, -3.0162356e+00, -2.6041422e+00,
       -1.8351980e+00, -2.1448617e+00, -2.2073743e+00, -1.6085958e+00,
       -1.3906698e+00, -2.2272210e+00, -9.0112889e-01, -1.5897094e+00,
       -5.5430478e-01, -1.1452208e-01, -1.9398988e+00,  3.6388639e-01,
       -1.0521647e+00, -2.5669742e+00, -5.8487523e-01, -1.7628891e+00],
      dtype=float32), array([0.5324763 , 0.49924928, 0.5525301 , 0.60082424, 0.63416857,
       0.6658493 , 0.67721945, 0.6906166 , 0.6706758 , 0.6296377 ,
       0.5645629 , 0.55850345], dtype=float32), array([5.13288498e-01, 1.96801260e-01, 1.00975215e+00, 2.91586471e+00,
       1.8026

In [32]:
val_features = val_df.apply(extract_features, axis=1)

features_val = []
for i in range(0, len(val_features)):
    features_val.append(np.concatenate((
        val_features[i][0],
        val_features[i][1], 
        val_features[i][2], 
        val_features[i][3],
        val_features[i][4]), axis=0))

X_val = np.array(features_val)

C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/4101981541.py:14: FutureWarning: Pass y=[-7.2857034e-08  2.0414098e-05  5.5091179e-05 ... -1.2138936e-03
 -1.1201693e-03  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/4101981541.py:14: FutureWarning: Pass y=[-0.0008255  -0.00108773 -0.00106642 ...  0.00378365  0.0034297
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/4101981541.py:14: FutureWarning: Pass y=[ 0.00226879  0.00273387  0.00220541 ... -0.00162064 -0.00114973
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.mels

In [33]:
test_features = test_df.apply(extract_features, axis=1)

features_test = []
for i in range(0, len(test_features)):
    features_test.append(np.concatenate((
        test_features[i][0],
        test_features[i][1], 
        test_features[i][2], 
        test_features[i][3],
        test_features[i][4]), axis=0))

X_test = np.array(features_test)

C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/4101981541.py:14: FutureWarning: Pass y=[0.00099488 0.00150995 0.0015521  ... 0.00399578 0.00335236 0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/4101981541.py:14: FutureWarning: Pass y=[-1.0050000e-03 -1.1443967e-03 -9.1314997e-04 ...  2.8472254e-04
 -1.6279630e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\alexp\AppData\Local\Temp/ipykernel_9500/4101981541.py:14: FutureWarning: Pass y=[ 0.0005451   0.00153135  0.00204929 ... -0.0051632  -0.00510599
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectro

In [34]:
X_train = np.array(features_train)
y_train = np.array(train_df['speaker'])
y_val = np.array(val_df['speaker'])

In [35]:

# Hot encoding y
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_val = to_categorical(lb.fit_transform(y_val))

In [36]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [47]:
print(len(X_train[0]))
print(X_val)

193
[[-0.45320253 -0.95751736 -0.59923078 ... -1.34973686  0.16624716
  -0.50628907]
 [-0.6825567   0.16729908  0.54109496 ... -0.39113406  0.88366633
  -0.65102639]
 [ 1.7023355   0.00708842 -0.96284691 ... -0.91259035  1.49470519
  -0.88367435]
 ...
 [-0.99147779 -0.60978403 -0.00666914 ... -0.52130704 -0.36602523
   0.2339812 ]
 [ 1.16322855 -0.25578316 -1.20772334 ... -0.25330933 -1.09342844
  -0.09395967]
 [-0.53236735  0.60548822  0.89420921 ...  2.50382377  1.3859789
  -0.04219378]]


In [37]:

# Build a simple dense model with early stopping and softmax for categorical classification, remember we have 30 classes
model = Sequential()
model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

In [ ]:
history = model.fit(X_train, y_train, batch_size=256, epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

Epoch 1/100


ValueError: in user code:

    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\compile_utils.py", line 201